In [13]:
import pandas as pd
import os as os
import numpy as np
import talib as talib
import json as json
pd.options.mode.chained_assignment = None

In [14]:
# 1.0
file = 't5-0608-0922-e10-u512-w50'
direction = 'short' # 方向
vol = 0.5 # 买卖波幅
vol2 = 0 # 止蚀波幅
cutloss = 1.05 # 止蚀
is_adjust = True # 调整
minutes = 5 # 预测x分钟后

# 1.1
path1 = os.path.abspath(os.path.join('data', 'nq', 'prediction', file+'.csv'))
df1 = pd.read_csv(path1)
df1.udate = pd.to_datetime(df1.udate)
df1.index = pd.to_datetime(df1.udate)
df1

,udate,High,Low,Open,Close,Volume,t1,t2,t3,t4,t5
udate,,,,,,,,,,,
2020-06-08 17:00:00,2020-06-08 17:00:00,9865.50,9865.50,9865.50,9865.50,0.0,NaN,NaN,NaN,NaN,NaN
2020-06-08 17:01:00,2020-06-08 17:01:00,9865.50,9865.50,9865.50,9865.50,0.0,NaN,NaN,NaN,NaN,NaN
2020-06-08 17:02:00,2020-06-08 17:02:00,9860.25,9860.50,9859.25,9859.50,5.0,NaN,NaN,NaN,NaN,NaN
2020-06-08 17:03:00,2020-06-08 17:03:00,9859.50,9860.00,9857.50,9857.50,12.0,NaN,NaN,NaN,NaN,NaN
2020-06-08 17:04:00,2020-06-08 17:04:00,9857.50,9859.25,9857.50,9859.25,1.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2020-09-22 15:49:00,2020-09-22 15:49:00,11149.75,11150.50,11149.50,11149.75,23.0,11160.756373,11160.998925,11161.330512,11162.469866,11160.782499
2020-09-22 15:50:00,2020-09-22 15:50:00,11149.75,11151.75,11148.50,11151.75,77.0,11160.650260,11160.898649,11161.228754,11162.370856,11160.659617
2020-09-22 15:51:00,2020-09-22 15:51:00,11151.75,11152.50,11150.75,11151.25,43.0,11161.214671,11161.443666,11161.781090,11162.924953,11161.205376


In [15]:
# 2.0
df2 = df1.copy(deep=True)

highs = np.array(df2['High'], dtype='float')
lows = np.array(df2['Low'], dtype='float')
opens = np.array(df2['Open'], dtype='float')
closes = np.array(df2['Close'], dtype='float')
vols = np.array(df2['Volume'], dtype='float')
df2['atr'] = talib.ATR(highs, lows, closes, timeperiod=14*2)

df2['last1'] = df2['Close'].shift(periods=-1) # 上笔数据 最后价
df2['adjust'] = df2['Close'] - df2['last1'] # 前1分钟 结算价 波幅

key2 = 't'+str(minutes)
if not is_adjust:
    df2['plast'] = df2[key2] # 预测 5分钟后的 波幅 
    df2['plast1'] = df2['t1'] # 预测 1分钟后的 波幅
else:
    df2['plast'] = df2[key2] + df2['adjust'] # 已调整
    df2['plast1'] = df2['t1'] + df2['adjust'] # 已调整

df2['pchange'] = (df2['plast'] - df2['Close']) / df2['Close']*100 # 预测 5分钟后的 波幅率
df2['pchange1'] = (df2['plast1'] - df2['Close']) / df2['Close']*100 # 预测 1分钟后的 波幅率

df2

,udate,High,Low,Open,Close,Volume,t1,t2,t3,t4,t5,atr,last1,adjust,plast,plast1,pchange,pchange1
udate,,,,,,,,,,,,,,,,,,
2020-06-08 17:00:00,2020-06-08 17:00:00,9865.50,9865.50,9865.50,9865.50,0.0,NaN,NaN,NaN,NaN,NaN,NaN,9865.50,0.00,NaN,NaN,NaN,NaN
2020-06-08 17:01:00,2020-06-08 17:01:00,9865.50,9865.50,9865.50,9865.50,0.0,NaN,NaN,NaN,NaN,NaN,NaN,9859.50,6.00,NaN,NaN,NaN,NaN
2020-06-08 17:02:00,2020-06-08 17:02:00,9860.25,9860.50,9859.25,9859.50,5.0,NaN,NaN,NaN,NaN,NaN,NaN,9857.50,2.00,NaN,NaN,NaN,NaN
2020-06-08 17:03:00,2020-06-08 17:03:00,9859.50,9860.00,9857.50,9857.50,12.0,NaN,NaN,NaN,NaN,NaN,NaN,9859.25,-1.75,NaN,NaN,NaN,NaN
2020-06-08 17:04:00,2020-06-08 17:04:00,9857.50,9859.25,9857.50,9859.25,1.0,NaN,NaN,NaN,NaN,NaN,NaN,9859.00,0.25,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-22 15:49:00,2020-09-22 15:49:00,11149.75,11150.50,11149.50,11149.75,23.0,11160.756373,11160.998925,11161.330512,11162.469866,11160.782499,2.762027,11151.75,-2.00,11158.782499,11158.756373,0.081011,0.080776
2020-09-22 15:50:00,2020-09-22 15:50:00,11149.75,11151.75,11148.50,11151.75,77.0,11160.650260,11160.898649,11161.228754,11162.370856,11160.659617,2.734812,11151.25,0.50,11161.159617,11161.150260,0.084378,0.084294
2020-09-22 15:51:00,2020-09-22 15:51:00,11151.75,11152.50,11150.75,11151.25,43.0,11161.214671,11161.443666,11161.781090,11162.924953,11161.205376,2.663926,11151.00,0.25,11161.455376,11161.464671,0.091518,0.091601


In [16]:
# 3.0
df4 = pd.DataFrame(columns = ['udate', 'action', 'cash', 'profit', 'message'])
holding = False # 是否持有仓位
top_profit_price = 0 # 买入点数
fee = 2.1 # 交易费
cash = 0

def is_trigger(v2, vol):
    if direction == 'long':
        return (v2['pchange'] >= vol)
    elif direction == 'short':
        return (v2['pchange'] <= -vol)

def is_cut_loss(v2, top_profit_price, direction):
    if direction == 'long':
        return (abs(v1['Close']-top_profit_price) >= abs(cutloss*v1['atr']))
    elif direction == 'short':
        return (abs(top_profit_price-v1['Close']) >= abs(cutloss*v1['atr']))
    
def is_stop_profit(v2, vol2):
    if direction == 'long':
        return (v2['pchange1'] <= vol2)
    elif direction == 'short':
        return (v2['pchange1'] >= vol2)

df3 = df2.copy(deep=True)
for k1, v1 in df3.iterrows():
    if v1['udate'].hour < 15 or v1['udate'].hour >= 19:
        # 3.2 买入
        if is_trigger(v1, vol) and not holding:
            holding = True
            cost1 = v1['Close']
            top_profit_price = v1['Close']
            action = 'buy' if direction == 'long' else 'sell'
            df4 = df4.append({'udate': k1, 'action': action, 'cash': np.nan, 'profit': np.nan, 'message': np.nan}, ignore_index=True)
            continue
        # 3.3
        cut_loss2 = is_cut_loss(v1, top_profit_price, direction) # 止蚀
        stop_profit2 = is_stop_profit(v1, vol2) # 止盈
        if (cut_loss2 or stop_profit2) and holding:
            holding = False
            profit2 = (v1['Close']-cost1)*20 - (fee*2)
            cash = cash + profit2
            if profit2 > 0:
                profit3 = '<span class="text-danger">+'+str(profit2)+'</span>'
            elif profit2 <= 0:
                profit3 = '<span class="text-success">'+str(profit2)+'</span>'
            message = '('+str(cost1)+' - '+str(v1['Close'])+')*20 - ('+str(fee)+'*2) = '+profit3
            # 3.4 长仓
            if direction == 'long' and stop_profit2:
                action = 'sell'
            elif direction == 'long' and cut_loss2:
                action = 'cut'
            # 3.5 短仓
            elif direction == 'short' and stop_profit2:
                action = 'buy'
            elif direction == 'short' and cut_loss2:
                action = 'cut'
            df4 = df4.append({'udate': k1, 'action': action, 'cash': cash, 'profit': profit2, 'message': message}, ignore_index=True)
        
df4.index = df4['udate']
df4.drop(['udate'], axis=1, inplace=True)
df4

,action,cash,profit,message
udate,,,,
2020-09-04 08:32:00,sell,NaN,NaN,NaN
2020-09-04 08:33:00,cut,305.8,305.8,(11801.0 - 11816.5)*20 - (2.1*2) = <span class...
2020-09-04 09:54:00,sell,NaN,NaN,NaN
2020-09-04 09:55:00,cut,986.6,680.8,(11287.75 - 11322.0)*20 - (2.1*2) = <span clas...
2020-09-04 09:56:00,sell,NaN,NaN,NaN
2020-09-04 09:58:00,cut,1352.4,365.8,(11338.25 - 11356.75)*20 - (2.1*2) = <span cla...
2020-09-04 10:33:00,sell,NaN,NaN,NaN
2020-09-04 10:34:00,cut,2073.2,720.8,(11314.75 - 11351.0)*20 - (2.1*2) = <span clas...
2020-09-04 10:35:00,sell,NaN,NaN,NaN


In [17]:
# 4.0
df5 = pd.concat([df2, df4], axis=1)
drop_list_5 = ['High', 'Low', 'Open', 'Volume', 't2', 't3', 't4', 'last1']
df5.drop(drop_list_5, axis=1, inplace=True)
df5

,udate,Close,t1,t5,atr,adjust,plast,plast1,pchange,pchange1,action,cash,profit,message
udate,,,,,,,,,,,,,,
2020-06-08 17:00:00,2020-06-08 17:00:00,9865.50,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-06-08 17:01:00,2020-06-08 17:01:00,9865.50,NaN,NaN,NaN,6.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-06-08 17:02:00,2020-06-08 17:02:00,9859.50,NaN,NaN,NaN,2.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-06-08 17:03:00,2020-06-08 17:03:00,9857.50,NaN,NaN,NaN,-1.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-06-08 17:04:00,2020-06-08 17:04:00,9859.25,NaN,NaN,NaN,0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-09-22 15:49:00,2020-09-22 15:49:00,11149.75,11160.756373,11160.782499,2.762027,-2.00,11158.782499,11158.756373,0.081011,0.080776,NaN,NaN,NaN,NaN
2020-09-22 15:50:00,2020-09-22 15:50:00,11151.75,11160.650260,11160.659617,2.734812,0.50,11161.159617,11161.150260,0.084378,0.084294,NaN,NaN,NaN,NaN
2020-09-22 15:51:00,2020-09-22 15:51:00,11151.25,11161.214671,11161.205376,2.663926,0.25,11161.455376,11161.464671,0.091518,0.091601,NaN,NaN,NaN,NaN


In [18]:
# 5.0
df6 = df5.loc[(df5['action']=='buy') | (df5['action']=='sell') | (df5['action']=='cut')]
df6.columns = ['udate', 'last', 'predict1(T+1)', 'predict1(T+5)', 'atr', 'shift', 'predict2(T+1)', 'predict2(T+5)', 'p-percent1', 'p-percent5', 'action', 'cash', 'profit', 'message']
df6['cash'] = df6['cash'].round(4)
df6['profit'] = df6['profit'].round(2)
# 5.1 
if direction == 'long':
    no_trade = df6.loc[(df6['action']=='buy')].shape[0]
elif direction == 'short':
    no_trade = df6.loc[(df6['action']=='sell')].shape[0]
total_profit = df6.iloc[-1]['cash'].round(2)
avg_profit = df6['profit'].mean().round(2)
max_win = df6['profit'].max().round(2)
max_loss = df6['profit'].min().round(2)
no_win = int(np.sum(df6['profit'] > 0))
no_loss = int(np.sum(df6['profit'] <= 0))
no_cut_loss = int(np.sum(df6['action'] == 'cut'))
no_total_day = len(df5.groupby(df5['udate'].dt.date).size())
no_trade_day = len(df5.groupby(df6['udate'].dt.date).size())
avg_no_trade = round(no_trade/no_total_day, 2)

# 5.2 加颜色
for k5, v5 in df6.iterrows():
    # 5.3 profit
    if df6.loc[k5, 'profit'] > 0:
        df6.loc[k5, 'profit'] = '<font class="text-danger">+'+str(df6.loc[k5, 'profit'])+'</font>'
    elif df6.loc[k5, 'profit'] <= 0:
        df6.loc[k5, 'profit'] = '<font class="text-success">'+str(df6.loc[k5, 'profit'])+'</font>'
    # 5.4 cash
    if df6.loc[k5, 'cash'] > 0:
        df6.loc[k5, 'cash'] = '<font class="text-danger">+'+str(df6.loc[k5, 'cash'])+'</font>'
    elif df6.loc[k5, 'cash'] <= 0:
        df6.loc[k5, 'cash'] = '<font class="text-success">'+str(df6.loc[k5, 'cash'])+'</font>'
html = df6.to_html(classes='table table-sm table-striped', index=False, escape=False, border=0).replace('border="1"', 'border="0"').replace('NaN', '')
data4 = {'total_profit': total_profit, 'avg_profit': avg_profit, 'max_win': max_win, 'max_loss': max_loss,
         'no_trade': no_trade, 'no_win': no_win, 'no_loss': no_loss, 'no_cut_loss': no_cut_loss, 'avg_no_trade': avg_no_trade, 'no_total_day': no_total_day,
         'html': html}
data5 = json.dumps(data4)

del data4['html']
print(data4)
df6

{'total_profit': 3881.4, 'avg_profit': 485.17, 'max_win': 720.8, 'max_loss': 305.8, 'no_trade': 8, 'no_win': 8, 'no_loss': 0, 'no_cut_loss': 8, 'avg_no_trade': 0.15, 'no_total_day': 55}


,udate,last,predict1(T+1),predict1(T+5),atr,shift,predict2(T+1),predict2(T+5),p-percent1,p-percent5,action,cash,profit,message
udate,,,,,,,,,,,,,,
2020-09-04 08:32:00,2020-09-04 08:32:00,11801.00,11753.056696,11754.133821,7.656638,-15.50,11738.633821,11737.556696,-0.528482,-0.537610,sell,NaN,NaN,NaN
2020-09-04 08:33:00,2020-09-04 08:33:00,11816.50,11759.598816,11760.577241,7.936758,25.50,11786.077241,11785.098816,-0.257460,-0.265740,cut,"<font class=""text-danger"">+305.8</font>","<font class=""text-danger"">+305.8</font>",(11801.0 - 11816.5)*20 - (2.1*2) = <span class...
2020-09-04 09:54:00,2020-09-04 09:54:00,11287.75,11254.080673,11254.431283,15.327050,-34.25,11220.181283,11219.830673,-0.598602,-0.601708,sell,NaN,NaN,NaN
2020-09-04 09:55:00,2020-09-04 09:55:00,11322.00,11262.883311,11263.409953,16.002870,-16.25,11247.159953,11246.633311,-0.661014,-0.665666,cut,"<font class=""text-danger"">+986.6</font>","<font class=""text-danger"">+680.8</font>",(11287.75 - 11322.0)*20 - (2.1*2) = <span clas...
2020-09-04 09:56:00,2020-09-04 09:56:00,11338.25,11272.912978,11273.612872,16.038482,6.50,11280.112872,11279.412978,-0.512752,-0.518925,sell,NaN,NaN,NaN
2020-09-04 09:58:00,2020-09-04 09:58:00,11356.75,11295.574641,11296.615541,16.069902,18.75,11315.365541,11314.324641,-0.364404,-0.373570,cut,"<font class=""text-danger"">+1352.4</font>","<font class=""text-danger"">+365.8</font>",(11338.25 - 11356.75)*20 - (2.1*2) = <span cla...
2020-09-04 10:33:00,2020-09-04 10:33:00,11314.75,11291.480311,11292.449347,13.449894,-36.25,11256.199347,11255.230311,-0.517472,-0.526036,sell,NaN,NaN,NaN
2020-09-04 10:34:00,2020-09-04 10:34:00,11351.00,11297.135111,11298.057143,14.282041,-1.75,11296.307143,11295.385111,-0.481833,-0.489956,cut,"<font class=""text-danger"">+2073.2</font>","<font class=""text-danger"">+720.8</font>",(11314.75 - 11351.0)*20 - (2.1*2) = <span clas...
2020-09-04 10:35:00,2020-09-04 10:35:00,11352.75,11305.279300,11306.173723,13.861254,-23.00,11283.173723,11282.279300,-0.612858,-0.620737,sell,NaN,NaN,NaN
